# Prediction using Neural Networks

In this notebook, we explore the provided data to build intuition on which models to use, which features to retain and more generally on the data challenge.

### Packages

In [ ]:
# os libraries
import time
import os

In [ ]:
# numerical libraries
import numpy as np
import pandas as pd

In [ ]:
# statistical learning libraries
import sklearn.ensemble as ens
import sklearn.feature_selection as fs
import sklearn.model_selection as ms
import sklearn.preprocessing as pr
import sklearn.linear_model as lm
import sklearn.neighbors as nb
import sklearn.svm as sv
import sklearn.neural_network as nn

In [ ]:
# neural networks libraries
import torch
from torch import nn

In [ ]:
# visualisation libraires
import matplotlib.pyplot as plt

### Functions

In [ ]:
def MSE(model, X, y):
    '''
    Get MSE of model on test data.
    
    Arguments:
        model: prediction model
        
    Returns:
        score: MSE loss
    '''
    
    # compute number of points in data
    n = y.shape[0]
    
    # return loss
    return (1/n) * np.sum(np.square(model.predict(X) - y))

In [ ]:
def export_results_nn(model, X):
    '''
    Export results into CSV file for submission.
    
    Arguments:
        model: regression model
    '''
    
    # obtain predictions
    pred = model(torch.tensor(X.values).float()).detach().numpy().squeeze()
    
    # obtain index of data
    idx = X.index
    
    # set in dataframe
    df_results = pd.DataFrame({'_ID': idx, '0': pred})
    
    # save dataframe
    df_results.to_csv('submissions/submit.csv', sep=',', index=False, index_label='_ID')

### Data Loading

In [ ]:
# read X_train
df_X_train = pd.read_csv('data/input_training.csv', sep=',', header=0, index_col=0)
X_train = df_X_train.values

In [ ]:
# read y_train
df_y_train = pd.read_csv('data/output_training.csv', sep=',', header=0, index_col=0)
y_train = df_y_train.values.ravel()

In [ ]:
# read X_test
df_X_test = pd.read_csv('data/input_testing.csv', sep=',', header=0, index_col=0)
X_test = df_X_test.values

### Data Normalisation

In [ ]:
# concatenate train and test datasets
df = pd.concat([df_X_train, df_X_test])

### Exploration and creation of an augmented dataset

In [ ]:
# create summary train dataset
summary = pd.DataFrame(columns=['Mean', 'Standard deviation', 'Range', 'Number of values', 'Values'], index=df.columns)

# create Pandas summary train dataset
summary_df = df.describe()

# compute statistics for each feature
for feature in df.columns:
    mean = summary_df[feature][1]
    std = summary_df[feature][2]
    min = summary_df[feature][3]
    max = summary_df[feature][7]
    values = set(df[feature])
    n_values = len(set(values))
    
    # populate dataset if n_values <= 10
    if n_values <= 50:
        summary.loc[feature] = pd.Series({'Mean':'{:0.2f}'.format(mean),\
                                          'Standard deviation':'{:0.2f}'.format(std),\
                                          'Range':'[{:0.2f}, {:0.2f}]'.format(min, max),\
                                          'Number of values':'{:0.0f}'.format(n_values),\
                                          'Values':', '.join(["{:0.2f}".format(x) for x in sorted(values)])})
        
    
    # populate dataset otherwise
    else:
        summary.loc[feature] = pd.Series({'Mean':'{:0.2f}'.format(mean),\
                                          'Standard deviation':'{:0.2f}'.format(std),\
                                          'Range':'[{:0.2f}, {:0.2f}]'.format(min, max),\
                                          'Number of values':'{:0.0f}'.format(n_values),\
                                          'Values':'NA'})

In [ ]:
summary

In [ ]:
# set list of categorical features
categorical_features = ['X3', 'X6', 'X11', 'X15', 'X16', 'X18', 'X19', 'X22', 'X28', 'X32', 'X33', 'X35', 'X36',
                        'X42', 'X49', 'X56', 'X58', 'X60', 'X62', 'X64', 'X68', 'X73', 'X74', 'X83', 'X86', 'X90',
                        'X104', 'X108', 'X109', 'X116', 'X117', 'X122', 'X130', 'X137', 'X139', 'X140', 'X141',
                        'X143', 'X144', 'X148', 'X149', 'X151', 'X162', 'X168', 'X169', 'X172', 'X174', 'X176',
                        'X177', 'X182', 'X184', 'X186', 'X187', 'X192', 'X193', 'X195', 'X196', 'X197', 'X199',
                        'X206', 'X209', 'X217', 'X219', 'X222', 'X231', 'X235', 'X238', 'X242', 'X246', 'X256',
                        'X260', 'X270', 'X275', 'X281', 'X285', 'X286', 'X291', 'X298', 'X301', 'X303', 'X304',
                        'X307', 'X308', 'X312', 'X314', 'X318', 'X330', 'X332', 'X336', 'X337', 'X338']

In [ ]:
# set list of categorical features with exactly two possible values
categorical_features_two = summary[summary['Number of values'].astype(int) == 2].index

In [ ]:
# set list of categorical features with strictly more than two possible values
categorical_features_more_than_two = [x for x in categorical_features if x not in categorical_features_two]

In [ ]:
# create augmented train dataset by one-hot encoding features with strictly more than two possible values
df_augmented = df.copy()
for feature in categorical_features_more_than_two:
    _ = pd.get_dummies(df[feature])
    _.columns = [feature+'-'+str(i) for i in range(1, len(_.columns)+1)]
    df_augmented = df_augmented.drop(feature, axis = 1)
    df_augmented = df_augmented.join(_)

In [ ]:
# truncate to retrieve df_X_train
df_X_train_augmented = df_augmented.truncate(before=None, after=df_X_train.shape[0])
X_train_augmented = df_X_train_augmented.values

# truncate to retrieve df_X_test
df_X_test_augmented = df_augmented.truncate(before=df_X_train.shape[0]+1, after=None)
X_test_augmented = df_X_test_augmented.values

In [ ]:
# create validation dataset
Xt, Xv, yt, yv = ms.train_test_split(X_train, y_train, test_size=0.15)

In [ ]:
# create validation augmented dataset
Xta, Xva, yta, yva = ms.train_test_split(X_train_augmented, y_train, test_size=0.15)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
Xta = torch.tensor(Xta).float().to(device)
yta = torch.tensor(yta.squeeze()).float().to(device)

Xva = torch.tensor(Xva).float().to(device)
yva = torch.tensor(yva.squeeze()).float().to(device)

Xt = torch.tensor(Xt).float().to(device)
yt = torch.tensor(yt.squeeze()).float().to(device)

Xv = torch.tensor(Xv).float().to(device)
yv = torch.tensor(yv.squeeze()).float().to(device)

### Feature selection

In [ ]:
# print shape of datasets
print('Train data shape:', Xt.shape)
print('Train data (augmented) shape:', Xta.shape)

### Prediction

In [ ]:
class GaussianNoise(nn.Module):
    def __init__(self, stddev):
        super().__init__()
        self.stddev = stddev

    def forward(self, din):
        if self.training:
            return din + torch.autograd.Variable(torch.randn(din.size()) * self.stddev)
        return din

In [ ]:
class NN(nn.Module):
    
    def __init__(self, m, sigma, d):
        
        super(NN, self).__init__()
    
        self.m = m
        self.sigma = sigma
        self.d = d
    
        self.noise1 = GaussianNoise(self.sigma)
        
        self.linear1 = nn.Linear(480, self.m)
        self.batchnorm1 = nn.BatchNorm1d(self.m)
        self.relu1 = nn.LeakyReLU(0.1)
        self.dropout1 = nn.Dropout(self.d)
        
        self.linear2 = nn.Linear(self.m, self.m // 2)
        self.batchnorm2 = nn.BatchNorm1d(self.m // 2)
        self.relu2 = nn.LeakyReLU(0.1)
        self.dropout2 = nn.Dropout(self.d)
        
        self.linear3 = nn.Linear(self.m // 2, 10)
        self.batchnorm3 = nn.BatchNorm1d(10)
        self.relu3 = nn.LeakyReLU(0.1)
        self.dropout3 = nn.Dropout(self.d)
        
        self.linear4 = nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.noise1(x)
        
        x = self.linear1(x)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        
        x = self.linear2(x)
        x = self.batchnorm2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        
        x = self.linear3(x)
        x = self.batchnorm3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        
        x = self.linear4(x)
        
        return x

In [ ]:
#Xta = torch.tensor(X_train_augmented).float().to(device)
#yta = torch.tensor(y_train.squeeze()).float().to(device)

In [ ]:
def train(model, lr, n_epochs, verbose=True):
        '''
        Training function.

        Arguments:
            model: torchvision.model
                - model to train

        Returns:
            model: torchvision.model
                - trained model
            losses: dict
                - training and validation losses (per epoch)
        '''

        if verbose == True:
            # print function start
            print('#'*30)
            print('Initialising training \n')

        # training start time
        time_start = time.time()

        # set loss function
        loss_function = nn.MSELoss()

        # set optimizer
        optimizer = torch.optim.RAdam(model.parameters(), weight_decay=1e-5, lr=lr, betas=[0.9, 0.99])

        # intialise lists to store validation and train accuracies and losses
        losses = {'train':[], 'validation':[]}

        # initialise past validation losses for LR scheduler
        past_validation_losses = [np.inf]*10
        
        if verbose == True:
            # print training start
            print('#'*30)
            print('Starting training\n')

        # iterate over epochs
        for epoch in range(n_epochs):

            # epoch start time
            time_start_epoch = time.time()
            
            if verbose == True:
                # print current epoch number
                print(('#' * (30)))
                print('Starting epoch {}/{}'.format(epoch+1, n_epochs))
                print(('-' * (30)))

            # iterate over train and validation phases
            for phase in ('train', 'validation'):

                # set model to training mode
                if phase == 'train':
                    model.train()

                # set model to evaluation mode for validation
                else:
                    model.eval()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    
                    # compute loss
                    if phase == 'train':
                        outputs = model(Xta)
                        loss = loss_function(outputs.squeeze(), yta)
                    
                    elif phase == 'validation':
                        outputs = model(Xva)
                        loss = loss_function(outputs.squeeze(), yva)
                    
                    # backward
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # print epoch's loss and accuracy at training phase
                if phase == "train":
                    if verbose == True:
                        print('Training phase | Loss: {:.5f}'.format(loss.item()))

                    # add loss to loss history
                    losses['train'].append(loss.item())

                # print epoch's loss and accuracy at validation phase, and update learning rate if required
                elif phase == 'validation':
                    if verbose == True:
                        print('Validat. phase | Loss: {:.5f}'.format(loss.item()))

                    # change learning rate
                    past_validation_losses.append(loss.item())
                    past_validation_losses.pop(0)

                    # check if validation loss is not decreasing anymore
                    if all(i <= past_validation_losses[-1] for i in past_validation_losses):

                        # update learning rate
                        lr = lr / 2

                        # update optimizer
                        optimizer = torch.optim.RAdam(model.parameters(), weight_decay=1e-5, lr=lr, betas=[0.9, 0.999])
                        
                        if verbose == True:
                            # printe learning rate update
                            print('*'*30)
                            print('Learning rate update to: {:.0e}'.format(lr))
                            print('*'*30)

                        # reset past validation losses
                        past_validation_losses = [np.inf]*10

                    # add loss to loss history
                    losses['validation'].append(loss.item())
                    
                    if verbose == True:
                        # plot losses
                        plt.figure(figsize=(12,8))
                        plt.plot(range(len(losses['train'])), losses['train'], label = 'Training Loss', color='black', linestyle='dashed')
                        plt.plot(range(len(losses['validation'])), losses['validation'], label = 'Validation Loss', color='black')
                        plt.legend()
                        plt.xlabel('Number of epochs')
                        plt.ylabel('Loss')
                        plt.savefig('loss.png')
                        plt.close()

            if verbose == True:
                # print time since start of epoch
                time_end_epoch = time.time()
                time_epoch = time_end_epoch - time_start_epoch
                print(('-' * (30)))
                print('Epoch complete in {:.0f}m {:.0f}s \n'.format(time_epoch // 60, time_epoch % 60))

        if verbose == True:
            # print time since start of epoch
            time_end = time.time()
            time_training = time_end - time_start
            print(('#' * (30)))
            print('Training complete in {:.0f}m {:.0f}s'.format(time_training // 60, time_training % 60))

        # return model and losses
        return model, losses

In [ ]:
def initialise(model, init_method='kaiming'):
    
    with torch.no_grad():
        
        if init_method == 'normal':
            for m in model.modules():
                if isinstance(m, nn.Linear) or isinstance(m, nn.Conv1d):
                    nn.init.normal_(m.weight.data)
                    m.bias.data.zero_()

        elif init_method == 'xavier':
            for m in model.modules():
                if isinstance(m, nn.Linear) or isinstance(m, nn.Conv1d):
                    nn.init.xavier_normal_(m.weight.data)
                    m.bias.data.zero_()

        elif init_method == 'kaiming':
            for m in model.modules():
                if isinstance(m, nn.Linear) or isinstance(m, nn.Conv1d):
                    nn.init.kaiming_normal_(m.weight.data)
                    m.bias.data.zero_()
                    
    return model

In [ ]:
model = NN(m=128, sigma=0.2, d=0.2)

In [ ]:
model

In [ ]:
model = initialise(model, init_method='kaiming')

In [ ]:
model, losses = train(model, lr=0.1, n_epochs=200)

In [ ]:
export_results_nn(model, df_X_test_augmented)